# Cliente PowerP API (Python)

Notebook de referencia en espanol para listar senales y consultar valores de la API en tiempo real usando el flujo de credenciales seguras (Client Credentials).

## Requisitos previos
- Variables de entorno: `POWERP_CLIENT_ID`, `POWERP_CLIENT_SECRET` y opcionalmente `POWERP_API_BASE_URL` (por defecto `https://tenant.powerp.app/rt-api/api`).
- Instalar dependencias: `pip install -r samples/python/requirements.txt`.
- Mantener bloques por debajo de 20 senales; 5-10 es ideal.

In [ ]:
import os
from collections import defaultdict
from datetime import datetime, timedelta
from typing import Any, Iterable, List
import requests
import pandas as pd

In [ ]:
# Configuracion basica
CLIENT_ID = os.environ.get("POWERP_CLIENT_ID")
CLIENT_SECRET = os.environ.get("POWERP_CLIENT_SECRET")
BASE_URL = os.environ.get("POWERP_API_BASE_URL", "https://tenant.powerp.app/rt-api/api").rstrip("/")
MAX_BLOCK_SIZE = 20  # limite de la API
DEFAULT_BLOCK_SIZE = min(10, MAX_BLOCK_SIZE)
LOOKBACK_MINUTES = 15  # ventanas de raw data debajo de 30 minutos
WINDOW_PERIOD = "200ms"

if not CLIENT_ID or not CLIENT_SECRET:
    raise RuntimeError("Configura POWERP_CLIENT_ID y POWERP_CLIENT_SECRET antes de ejecutar el notebook")

def obtener_token_acceso():
    try:
        print("Autenticando...")
        res = requests.post(f"{BASE_URL}/v1/auth/token", json={
            "clientId": CLIENT_ID,
            "clientSecret": CLIENT_SECRET
        }, timeout=10)
        res.raise_for_status()
        token = res.json()["accessToken"]
        print(f"Autenticado! Longitud del token: {len(token)}")
        return token
    except Exception as e:
        raise RuntimeError(f"Fallo la autenticacion: {e}")

TOKEN = obtener_token_acceso()

HEADERS = {
    "Authorization": f"Bearer {TOKEN}",
    "Accept": "application/json",
}

In [ ]:
def chunked(items: List[Any], size: int) -> Iterable[List[Any]]:
    size = max(1, min(size, MAX_BLOCK_SIZE))
    for idx in range(0, len(items), size):
        yield items[idx : idx + size]


def listar_senales() -> List[dict]:
    response = requests.get(f"{BASE_URL}/v1/measurements", headers=HEADERS, timeout=30)
    response.raise_for_status()
    return response.json()


def consultar_valores(database_id: int, indexes: List[str], start_time: datetime, end_time: datetime, agg_function: str, window_period: str = WINDOW_PERIOD) -> List[dict]:
    payload = {
        "databaseId": database_id,
        "measurementIndexes": indexes,
        "startTime": start_time.isoformat() + "Z",
        "endTime": end_time.isoformat() + "Z",
        "aggFunction": agg_function,
        "windowPeriod": window_period,
    }
    response = requests.post(f"{BASE_URL}/v1/Query", headers=HEADERS, json=payload, timeout=30)
    response.raise_for_status()
    return response.json()


print("Configuracion lista; consultando metadatos...")

In [ ]:
# Listar senales y consultar en bloques seguros
mediciones = listar_senales()
print(f"Total de senales: {len(mediciones)}")

agrupadas = defaultdict(list)
for fila in mediciones:
    llave = (fila.get("databaseId"), fila.get("defaultAgg"))
    agrupadas[llave].append(fila)

fin = datetime.utcnow()
inicio = fin - timedelta(minutes=LOOKBACK_MINUTES)
block_size = DEFAULT_BLOCK_SIZE

for (database_id, agg_function), filas in agrupadas.items():
    agg = agg_function or "last"
    print(f"\nBase {database_id} | Agg {agg} | Senales {len(filas)}")
    for lote in chunked(filas, block_size):
        indices = [str(item["index"]) for item in lote]
        datos = consultar_valores(database_id, indices, inicio, fin, agg)
        print(f"- Bloque {len(indices)} -> {len(datos)} puntos recibidos")
        if datos:
            muestra = datos[0]
            print(f"  muestra: index={muestra.get('index')} valor={muestra.get('value')} ts={muestra.get('timestamp')}")

## Buenas practicas
- Usa bloques de 5 a 10 senales; nunca excedas 20.
- Mantén ventanas de raw data por debajo de 30 minutos (usa 15 como base).
- Evita registrar tokens o payloads completos en logs.
- Ante respuestas 429/5xx aplica backoff y reduce el tamano de bloque.

## Solo metadatos
Lista las senales sin consultar valores; util para descubrir indices y agregaciones por defecto.

In [ ]:
# Listar metadatos sin llamar al endpoint de datos
mediciones = listar_senales()
print(f"Total de senales: {len(mediciones)}")

if mediciones:
    df = pd.DataFrame(mediciones)[[
        "index", "name", "databaseId", "defaultAgg", "dataType", "unitSymbol", "firstDataPoint", "minValue", "maxValue"
    ]]
    print("Vista previa de metadatos (primeras 20 filas):")
    print(df.head(20))
else:
    print("No se recibieron senales.")


## Consultar indices seleccionados
Ingresa los indices que quieras revisar manualmente; usa ventanas pequenas (<=30 minutos).

In [ ]:
# Reemplaza con los indices que quieras consultar (strings)
indices_manuales = ["2098", "67634"]  # ej. ["101", "205"]

# Usa metadatos para inferir base de datos y agregacion
if 'mediciones' not in globals() or not mediciones:
    mediciones = listar_senales()

database_id = mediciones[0].get("databaseId") if mediciones else None
agg_function = (mediciones[0].get("defaultAgg") if mediciones else None) or "last"
print(f"Consultando Base {database_id} con Agg {agg_function}")

fin = datetime.utcnow()
inicio = fin - timedelta(minutes=LOOKBACK_MINUTES)

if not database_id:
    raise ValueError("Define database_id manualmente o asegúrate de cargar las mediciones.")

datos = consultar_valores(database_id, indices_manuales, inicio, fin, agg_function)
print(f"Solicitados {indices_manuales} -> {len(datos)} puntos recibidos")
if datos:
    for item in datos:
        print(f"idx={item.get('index')} valor={item.get('value')} ts={item.get('timestamp')}")
